# Generate interval best execution benchmarks using dedicated service - Python

### Overview
Best execution aims to measure the performance of executed trades, an execution algorithm or an execution venue.

This sample is based on an `interval` approach and designed to illustrate market activity over a look back period.

<div class="alert alert-block alert-info">
    <b>Note:</b> To explore the point-in-time approach, suitable to highlight individual tradres among market activity. Please refer to <i><a href="">bestex-pointintime</a></i> notebook.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** best execution *interval* benchmarks.

### Inputs/outputs
Best execution interval sample requires instrument's identifier, date time intervals as per inputs. Results are as follows:
* a snapshot of occured trades
* a set of computed benchmarks based on the occured trades

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *matpotlib* as per display package
    
***

# Run BestEx interval sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib

In [ ]:
pip install systemathics.apis --pre

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_trades_pb2 as tick_trades
import systemathics.apis.services.tick.v1.tick_trades_pb2_grpc as tick_trades_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

start = datetime(2021, 11, 15, 16, 20,0)
end = datetime(2021, 11, 15, 16, 30,0)

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = start.year, month = start.month, day = start.day), 
    end_date = date.Date(year = end.year, month = end.month, day = end.day)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = start.hour, minutes = start.minute, seconds = start.second), 
    end_time = timeofday.TimeOfDay(hours = end.hour, minutes = end.minute, seconds = end.second)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
# generate constraints based on the previous time selection
my_constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

In [ ]:
# generate the tick trades request
my_identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
request = tick_trades.TickTradesRequest(
    identifiers = [my_identifier],
    constraints = my_constraints
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  

        # instantiate the tick trades service
        service = tick_trades_service.TickTradesServiceStub(channel)
        
        # process the tick trades request
        trades_data = []
        metadata = [('authorization', token)]
        for trade in service.TickTrades(request=request, metadata=metadata):
            if trade.data:
                trades_data.append(trade)

    # keep only trades, delete instrument mapping (1st element retrieved)
    # mapping is not used as only one instrument identifier is requested
    del trades_data[0]
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

The following code snippet displays computed benchmarks for the given input time window:

In [ ]:
# create the data frame
dates=[datetime.fromtimestamp(t.data.time_stamp.seconds) for t in trades_data]
prices=[t.data.trade.price for t in trades_data]
sizes=[t.data.trade.size for t in trades_data]
d = {'Date': dates, 'Size': sizes, 'Price': prices}
df = pd.DataFrame(data=d)
df 

In [ ]:
# visualize interval benchmarks
vwap = sum([t.data.trade.price * t.data.trade.size for t in trades_data]) / sum(sizes) 

print('Interval start time \t: {}'.format(start))
print('Interval end time \t: {}'.format(end))
print('Last trade occured at \t: {}'.format(dates[-1]))
print('Total trades retrieved \t: {}'.format(len(trades_data)))
print('Cumulated trade sizes \t: {}'.format(sum(sizes)))
print('Max trade price \t: {}'.format(max(prices)))
print('Min trade price \t: {}'.format(min(prices)))
print('Average trade price \t: {:.2f}'.format(sum(prices) / len(prices)))
print('Average trade size \t: {:.2f}'.format(sum(sizes) / len(sizes)))
print('Max trade size \t\t: {}'.format(max(sizes)))
print('Min trade size \t\t: {}'.format(min(sizes)))
print('Vwap \t\t\t: {:.2f}'.format(vwap))

### Step 4: Visualize data

#### 4.1 Plot trades with matplotlib

In [ ]:
import matplotlib.pyplot as plt

# create figure and axis objects with subplots()
fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.plot('Date', 'Price', data=df, marker='', color='red', linewidth=2, label="Price")
ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="red",fontsize=14)

# twin object for two different y-axis on the sample plot
ax2=ax.twinx()
#ax2.bar('Date','Size',data=df, align='center', alpha=0.2, linewidth=0.1)
ax2.plot('Date', 'Size', data=df, marker='', color='purple', linewidth=2, label="Price")
ax2.set_ylabel("Size",color="purple",fontsize=14)
plt.title('Trades for {0}-{1} | Trades count: {2} | Toal traded volume: {3}'.format(ticker, exchange, len(trades_data), sum(sizes)))
plt.show()